In [1]:
import numpy as np
import pprint as pp
import shutil
import utils2p.synchronization
import yaml
from pathlib import Path
from rye2p.thorimage.settings import FunctionalImagingSettings

# set upstairs or downstairs microscope
microscope = 'downstairs-2p'

# set project directory paths
PRJ_DATA_DIR = Path("/local/tensor/Ruoyi/ruoyi_mbc1_project")
RAW_DATA_DIR = PRJ_DATA_DIR / "raw_data"
NAS_PROC_DIR = PRJ_DATA_DIR / "processed_data"

# ThorImage pre-processing demo

We convert .raw ThorImage movies into more easily readable .tif stacks, and save them to a processed data folder.

The Experiment.xml file is also copied to the processed data folder, and important information about the ThorImage
recording is saved to `thorimage_settings.yaml`.

# Set dataset info

ThorImage and ThorSync files are assumed to be stored in `RAW_DATA_DIR` in directories with the following structure:
```
raw_data/
+-- {{date_imaged}}
    +-- {{fly_num}}
        +-- {{thorimage_name}}
        +-- {{thorsync_name}}
```

In [2]:
date_imaged = '2025-10-12'
fly_num = 'LH1900xG8f;G8f_5d'
thorimage_name = 'HANG_L'
thorsync_name = 'SyncData002'

## Get the ThorImage and ThorSync folders

- `sync_mov_dir`: ThorSync directory
- `raw_mov_dir`: ThorImage directory
- `proc_mov_dir`: Processed data directory (holds outputs from processing the ThorImage and ThorSync files)


In [4]:
# ThorImage folder containing Experiment.xml and .raw movie
raw_mov_dir = RAW_DATA_DIR.joinpath(date_imaged,
                                      str(fly_num),
                                      thorimage_name)

# folder to save the processed movie outputs
proc_mov_dir = NAS_PROC_DIR.joinpath(date_imaged, str(fly_num), thorimage_name)

# ThorSync folder containing 'Episode001.h5' and 'ThorImageSettings.xml'
sync_dir = RAW_DATA_DIR.joinpath(date_imaged,
                                      str(fly_num),
                                      thorsync_name)

print(f"raw_mov_dir: {raw_mov_dir}")
print(f"proc_mov_dir: {proc_mov_dir}")
print(f"sync_dir: {sync_dir}")

raw_mov_dir: /local/tensor/Ruoyi/ruoyi_mbc1_project/raw_data/2025-10-12/LH1900xG8f;G8f_5d/HANG_L
proc_mov_dir: /local/tensor/Ruoyi/ruoyi_mbc1_project/processed_data/2025-10-12/LH1900xG8f;G8f_5d/HANG_L
sync_dir: /local/tensor/Ruoyi/ruoyi_mbc1_project/raw_data/2025-10-12/LH1900xG8f;G8f_5d/SyncData002


## Get the ThorImage metadata

ThorImage metadata is parsed from the `Experiment.xml` file.

In [5]:
thorimage_xml = RAW_DATA_DIR.joinpath(date_imaged,
                                      str(fly_num),
                                      thorimage_name,
                                      'Experiment.xml')

# Copy Experiment.xml to proc_mov_dir
proc_mov_dir.mkdir(parents=True, exist_ok=True)
shutil.copy(thorimage_xml, proc_mov_dir.joinpath("Experiment.xml"))

# Get the relevant thorimage settings
thorimage_settings = FunctionalImagingSettings.from_xml(thorimage_xml)
thorimage_settings.thorsync_name = thorsync_name

# Save thorimage_settings to yaml file in proc_mov_dir
thorimage_settings.to_yaml(proc_mov_dir / "thorimage_settings.yaml")

pp.pprint(thorimage_settings.to_dict(), indent=4, sort_dicts=False)


{   'raw_frame_rate': 3.663,
    'frame_rate': 0.30524999999999997,
    'n_frames': 6576,
    'n_timepoints': 548,
    'fast_z': True,
    'n_z': 11,
    'n_flyback_frames': 1,
    'step_size_um': 5.0,
    'average_mode': False,
    'n_averaging': 1,
    'steps_per_frame': 12,
    'timelapse_trigger_mode': 0,
    'streaming_trigger_mode': 4,
    'stimulus_triggering': 1,
    'software_version': '3.0.2016.10131',
    'experiment_notes': '20251012_122101_stimuli.yaml',
    'thorsync_name': 'SyncData002'}


# Convert .raw movie to .tif stacks

## Load the .raw movie


In [6]:
meta = utils2p.Metadata(thorimage_xml)

raw_file =raw_mov_dir.joinpath("Image_0001_0001.raw")

# Load movie
stack1, = utils2p.load_raw(raw_file, meta)
stack1.shape


(548, 11, 192, 192)

## Save as single .tif file in `proc_mov_dir`


In [7]:
# convert to relative path
tiff_file = proc_mov_dir.joinpath("Image_0001_0001.tif")

print(f"saving: {tiff_file}")
utils2p.save_img(tiff_file, stack1)
print(f"...saved successfully.")

saving: /local/tensor/Ruoyi/ruoyi_mbc1_project/processed_data/2025-10-12/LH1900xG8f;G8f_5d/HANG_L/Image_0001_0001.tif
...saved successfully.


## Save as sequence of .tif stacks
The .tif sequence is saved in a subfolder `stk/`. This helps for faster loading in ImageJ and other applications.

Set the # of timepoints per .tif file in `frames_per_batch`.

```
{{proc_mov_dir}}/
--+ Experiment.xml
--+ Image_001_001.tif
--+ stk/
    --+ stk_001.tif
    --+ stk_002.tif
    --+ ...
```

In [23]:
stk_dir = proc_mov_dir.joinpath('stk')
stk_dir.mkdir(parents=True, exist_ok=True)

frames_per_batch = 500

batch_gen = utils2p.load_stack_batches(tiff_file, frames_per_batch)

print(stk_dir)

for i, batch in enumerate(batch_gen):
    save_name = stk_dir.joinpath(f"stk_{i:03d}.tif")
    utils2p.save_img(save_name, batch)
    print(f"\t- {save_name.name} saved successfully")

/local/tensor/Haoru/rotation_project/processed_data/2025-02-10/stk
	- stk_000.tif saved successfully
	- stk_001.tif saved successfully
	- stk_002.tif saved successfully


## Get the ThorSync metadata
```
{{sync_dir}}/
--+ Episode001.h5: contains the recorded ThorSync data
--+ ThorRealTimeDataSettings.xml: contains ThorSync metadata
```

In [5]:
# ThorImage file containing the input lines
#   - note: Sometimes the file is named 'Episode0001.h5'
h5_file = sync_dir.joinpath('Episode001.h5')

sync_meta_file = h5_file.with_name('ThorRealTimeDataSettings.xml')
sync_meta = utils2p.synchronization.SyncMetadata(h5_file.with_name("ThorRealTimeDataSettings.xml"))


print('\n--------------------------------------------------')
print(thorimage_xml.relative_to(RAW_DATA_DIR))
print('--------------------------------------------------')

pp.pprint(thorimage_settings.to_dict(), sort_dicts=False)

# %% Get ThorImage and ThorSync metadata

# ThorImage metadata
meta = utils2p.Metadata(thorimage_xml)

# ThorSync metadata


--------------------------------------------------
2025-10-12/LH1900xG8f;G8f_5d/HANG_L/Experiment.xml
--------------------------------------------------
{'raw_frame_rate': 3.663,
 'frame_rate': 0.30524999999999997,
 'n_frames': 6576,
 'n_timepoints': 548,
 'fast_z': True,
 'n_z': 11,
 'n_flyback_frames': 1,
 'step_size_um': 5.0,
 'average_mode': False,
 'n_averaging': 1,
 'steps_per_frame': 12,
 'timelapse_trigger_mode': 0,
 'streaming_trigger_mode': 4,
 'stimulus_triggering': 1,
 'software_version': '3.0.2016.10131',
 'experiment_notes': '20251012_122101_stimuli.yaml',
 'thorsync_name': 'SyncData001'}
